This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

# Project: Personalized Real Estate Agent

This project is the final submission for the Udacity Generative AI Course

## Step 1: Python Setup

In [25]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["ANONYMIZED_TELEMETRY"] = "false"

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""
if "OPENAI_API_BASE" not in os.environ:
    os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [26]:
from typing import List
from pydantic import BaseModel
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chains.question_answering import load_qa_chain

## Step 2: Generating Real Estate Listings

In [27]:
class Listing(BaseModel):
    neighborhood: str
    price: str
    bedrooms: int
    bathrooms: int
    house_size: str
    description: str
    neighborhood_description: str

class ListingList(BaseModel):
    listings: List[Listing]

In [28]:
parser = PydanticOutputParser(pydantic_object=ListingList)

prompt = PromptTemplate(
    template=(
        "You are a helpful assistant that generates realistic real estate listings.\n"
        "Generate {num_listings} real estate listings for {city}. "
        "Use descriptive language. Output must follow this schema:\n"
        "{format_instructions}"
    ),
    input_variables=["num_listings", "city", "format_instructions"],
)

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
)

chain = LLMChain(
    llm=model,
    prompt=prompt,
    output_parser=parser
)

generated_listings = chain.run({
    "num_listings": 20,
    "city": "San Francisco",
    "format_instructions": parser.get_format_instructions()
})

print(generated_listings)

listings=[Listing(neighborhood='Pacific Heights', price='$2,500,000', bedrooms=3, bathrooms=2, house_size='2,000 sqft', description='Beautiful Victorian home with stunning views of the Golden Gate Bridge.', neighborhood_description='Upscale neighborhood known for its historic architecture and panoramic views.'), Listing(neighborhood='Mission District', price='$1,200,000', bedrooms=2, bathrooms=1, house_size='1,100 sqft', description='Modern condo with open floor plan and rooftop deck.', neighborhood_description='Vibrant neighborhood with diverse culture and vibrant nightlife.'), Listing(neighborhood='Noe Valley', price='$3,000,000', bedrooms=4, bathrooms=3, house_size='2,500 sqft', description='Luxury single-family home with gourmet kitchen and backyard oasis.', neighborhood_description='Family-friendly area with trendy shops and cafes.'), Listing(neighborhood='Russian Hill', price='$1,800,000', bedrooms=2, bathrooms=2, house_size='1,600 sqft', description='Charming cottage with bay wi

## Step 3: Storing Listings in a Vector Database

In [30]:
listings = generated_listings.listings if hasattr(generated_listings, "listings") else generated_listings

docs = [
    Document(
        page_content=(
            f"{l.neighborhood} | {l.price} | {l.bedrooms} BR / {l.bathrooms} BA | {l.house_size}\n"
            f"{l.description}\n"
            f"Neighborhood: {l.neighborhood_description}"
        ),
        metadata={
            "neighborhood": l.neighborhood,
            "price": l.price,
            "bedrooms": l.bedrooms,
            "bathrooms": l.bathrooms,
            "house_size": l.house_size,
        },
    )
    for l in listings
]

# Embeddings & Vectorstore
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embeddings, collection_name="real_estate_listings")

Failed to send telemetry event client_start: capture() takes 1 positional argument but 3 were given


# Step 4: Building the User Preference Interface

In [32]:
# From the Instruction of the Udemy Course
questions = [   
                "How big do you want your house to be?", 
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."]

# Build user interaction string (question, answer, question, answer...)
interaction_parts = []
for q, a in zip(questions, answers):
    interaction_parts.append(f"Q: {q}")
    interaction_parts.append(f"A: {a}")

user_interaction = "\n".join(interaction_parts)



pref_prompt = PromptTemplate(
    template=(
        "Summarize the following buyer Q/A into a short buyer profile (3-5 bullet points).\n\n"
        "{user_interaction}"
    ),
    input_variables=["user_interaction"],
)

buyer_profile = LLMChain(llm=model, prompt=pref_prompt).run(
    user_interaction=user_interaction
)
print("Buyer profile:\n", buyer_profile)

Buyer profile:
 - Looking for a comfortable three-bedroom house with spacious kitchen and cozy living room
- Prioritizes quiet neighborhood, good local schools, and convenient shopping options
- Desires amenities such as backyard for gardening, two-car garage, and modern heating system
- Values easy access to reliable bus line, proximity to major highway, and bike-friendly roads
- Seeks a neighborhood that balances suburban tranquility with access to urban amenities like restaurants and theaters


## Step 5: Searching Based on Preferences

In [33]:
# Similarity Search mit der User-Interaktion als Query
suitable_listings = db.similarity_search(buyer_profile, k=3)

print(f"\n{'='*80}")
print(f"Found {len(suitable_listings)} suitable listings:")
print('='*80)

for i, doc in enumerate(suitable_listings, start=1):
    print(f"\n[Listing {i}]")
    print(f"Neighborhood: {doc.metadata.get('neighborhood', 'N/A')}")
    print(f"Price: {doc.metadata.get('price', 'N/A')}")
    print(f"Bedrooms: {doc.metadata.get('bedrooms', 'N/A')} | Bathrooms: {doc.metadata.get('bathrooms', 'N/A')}")
    print(f"Size: {doc.metadata.get('house_size', 'N/A')}")
    print(f"\nDescription:\n{doc.page_content[:200]}...")
    print('-'*80)


Found 3 suitable listings:

[Listing 1]
Neighborhood: Noe Valley
Price: $3,000,000
Bedrooms: 4 | Bathrooms: 3
Size: 2,500 sqft

Description:
Noe Valley | $3,000,000 | 4 BR / 3 BA | 2,500 sqft
Luxury single-family home with gourmet kitchen and backyard oasis.
Neighborhood: Family-friendly area with trendy shops and cafes....
--------------------------------------------------------------------------------

[Listing 2]
Neighborhood: Noe Valley
Price: $1,800,000
Bedrooms: 3 | Bathrooms: 2
Size: 1,800 sqft

Description:
Noe Valley | $1,800,000 | 3 BR / 2 BA | 1,800 sqft
Contemporary remodeled home with an open floor plan, high ceilings, and modern finishes. Private backyard with a deck perfect for entertaining.
Neigh...
--------------------------------------------------------------------------------

[Listing 3]
Neighborhood: Bernal Heights
Price: $2,100,000
Bedrooms: 3 | Bathrooms: 2
Size: 1,900 sqft

Description:
Bernal Heights | $2,100,000 | 3 BR / 2 BA | 1,900 sqft
Contemporary home wi

## Step 6: Personalizing Listing Descriptions

In [36]:
augment_prompt = PromptTemplate(
    template=(
        "You are a professional home seller and expert copywriter. Your goal is to make every listing sound as appealing as possible, while staying fully factual.\n"
        "RULES:\n"
        "- Do NOT give out the facts such as price, sqft, number of rooms.\n"
        "- Do NOT invent anything that is not already in the listing.\n"
        "- You MAY rephrase, reorder, and emphasize parts that match the buyer.\n"
        "- Use vivid, positive language and highlight unique features.\n"
        "- Make the description emotionally engaging and persuasive, as if you were selling the home to a friend.\n"
        "BUYER PROFILE:\n{buyer_profile}\n"
        "ORIGINAL LISTING:\n{listing}\n"
        "Rewritten listing:"
    ),
    input_variables=["buyer_profile", "listing"],
)

augment_chain = LLMChain(llm=model, prompt=augment_prompt)

for i, doc in enumerate(suitable_listings, start=1):
    augmented = augment_chain.run(
        buyer_profile=buyer_profile,
        listing=doc.page_content,
    )
    print(f"\n[Listing {i}]")
    print(f"Neighborhood: {doc.metadata.get('neighborhood', 'N/A')}")
    print(f"Price: {doc.metadata.get('price', 'N/A')}")
    print(f"Bedrooms: {doc.metadata.get('bedrooms', 'N/A')} | Bathrooms: {doc.metadata.get('bathrooms', 'N/A')}")
    print(f"Size: {doc.metadata.get('house_size', 'N/A')}")
    print(f"\nDescription:\n{augmented.strip()}") # Here the augmented description is used
    print('-'*80)


[Listing 1]
Neighborhood: Noe Valley
Price: $3,000,000
Bedrooms: 4 | Bathrooms: 3
Size: 2,500 sqft

Description:
Welcome to the charming neighborhood of Noe Valley, where this luxurious single-family home awaits you. Step inside to find a spacious gourmet kitchen, perfect for preparing delicious meals and hosting gatherings with loved ones. The cozy living room is ideal for relaxing after a long day, while the backyard oasis provides a peaceful retreat for gardening or enjoying the fresh air.

Located in a family-friendly area, this home is surrounded by trendy shops and cafes, offering the perfect blend of suburban tranquility and urban convenience. With easy access to a reliable bus line and proximity to major highways, commuting is a breeze. Plus, the neighborhood boasts bike-friendly roads for those who prefer to cycle.

Don't miss the two-car garage for convenient parking and the modern heating system for ultimate comfort. This home is perfect for those seeking a comfortable thre